![Add a relevant banner image here](path_to_image)

# Project Title

## Overview

This project seeks to build machine learning models to achieve two things:

1. Predict the highest ranking a song will achieve on the Billboard Hot 100 list.
2. Predict the largest week over week increase in a given song's ranking on the Hot 100 list.

Key Insights:

- A song's genre has a minimal impact on its ranking on the Billboard Hot 100 list.
- Song characteristics such as tempo, danceability, etc. appear to be the strongest drivers of their performance on the Hot 100 list.
- Analyzing music can quickly lead to an overwhelming number of features, so thoughtful and careful data selection is key.

## Business Understanding

The customer of this project is FutureProduct Advisors, a consultancy that helps their customers develop innovative and new consumer products. FutureProduct’s customers are increasingly seeking help from their consultants in go-to-market activities. 

FutureProduct’s consultants can support these go-to-market activities, but the business does not have all the infrastructure needed to support it. Their biggest ask is for a tool to help them find interesting, up-and-coming music to accompany social posts and online ads for go-to-market promotions. 

**Stakeholders**

- FutureProduct Managing Director: oversees their consulting practice and is sponsoring this project.
- FutureProduct Senior Consultants: the actual users of the prospective tool. A small subset of the consultants will pilot the prototype tool.
- My consulting leadership: sponsors of this effort; will provide oversight and technical input of the project as needed.

**Primary Goals**

1.	Build a data tool that can evaluate any song in the Billboard Hot 100 list and make predictions about:
    -	The song’s position on the Hot 100 list 4 weeks in the future
    -	The song’s highest position on the list in the next 6 months
2.	Create a rubric that lists the 3 most important factors for songs’ placement on the Hot 100 list for each hear from 2000 to 2021.


## Data Understanding

Billboard Hot 100 weekly charts (Kaggle): https://www.kaggle.com/datasets/thedevastator/billboard-hot-100-audio-features

I’ve chosen this dataset because it has a direct measurement of song popularity (the Hot 100 list) and because its long history gives significant context to a song’s positioning in a given week.
The features list gives a wide range of song attributes to explore and enables me to determine what features most significantly contribute to a song’s popularity and how that changes over time.


In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import ast
from collections import Counter

import xgboost as xgb

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay, mean_squared_error, r2_score, pairwise_distances

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

np.random.seed(42)

In [2]:
df_hotlist_all = pd.read_csv('Data/Hot Stuff.csv')
df_features_all = pd.read_csv('Data/Hot 100 Audio Features.csv')

In [3]:
# exploring hotlist df
df_hotlist_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327895 entries, 0 to 327894
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   index                   327895 non-null  int64  
 1   url                     327895 non-null  object 
 2   WeekID                  327895 non-null  object 
 3   Week Position           327895 non-null  int64  
 4   Song                    327895 non-null  object 
 5   Performer               327895 non-null  object 
 6   SongID                  327895 non-null  object 
 7   Instance                327895 non-null  int64  
 8   Previous Week Position  295941 non-null  float64
 9   Peak Position           327895 non-null  int64  
 10  Weeks on Chart          327895 non-null  int64  
dtypes: float64(1), int64(5), object(5)
memory usage: 27.5+ MB


In [4]:
# exploring features df
df_features_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29503 entries, 0 to 29502
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   index                      29503 non-null  int64  
 1   SongID                     29503 non-null  object 
 2   Performer                  29503 non-null  object 
 3   Song                       29503 non-null  object 
 4   spotify_genre              27903 non-null  object 
 5   spotify_track_id           24397 non-null  object 
 6   spotify_track_preview_url  14491 non-null  object 
 7   spotify_track_duration_ms  24397 non-null  float64
 8   spotify_track_explicit     24397 non-null  object 
 9   spotify_track_album        24391 non-null  object 
 10  danceability               24334 non-null  float64
 11  energy                     24334 non-null  float64
 12  key                        24334 non-null  float64
 13  loudness                   24334 non-null  flo

#### Exploratory Data Analysis

In [ ]:
liveness_dist = df_cleaned_genre['liveness'].value_counts()
df_liveness_dist = pd.DataFrame(liveness_dist)
df_liveness_dist = df_liveness_dist.reset_index()

plt.bar(df_liveness_dist['liveness'], df_liveness_dist['count'], color='orange')
plt.xlabel('Liveness Rating')
plt.ylabel('Number of Songs')
plt.title('Distribution of Liveness Ratings')
plt.show()

In [ ]:
danceability_dist = df_cleaned_genre['danceability'].value_counts()
df_danceability_dist = pd.DataFrame(danceability_dist)
df_danceability_dist = df_danceability_dist.reset_index()

plt.bar(df_danceability_dist['danceability'], df_danceability_dist['count'], color='skyblue')
plt.xlabel('Danceability Rating')
plt.ylabel('Number of Songs')
plt.title('Distribution of Danceability Ratings')
plt.show()

In [ ]:
acousticness_dist = df_cleaned_genre['acousticness'].value_counts()
df_acousticness_dist = pd.DataFrame(acousticness_dist)
df_acousticness_dist = df_acousticness_dist.reset_index()

plt.bar(df_acousticness_dist['acousticness'], df_acousticness_dist['count'], color='orange')
plt.xlabel('Acousticness Rating')
plt.ylabel('Number of Songs')
plt.title('Distribution of Acousticness Ratings')
plt.show()

In [ ]:
peak_pos_dist = df_cleaned_genre['Max_Peak_Position'].value_counts()
df_peak_pos_dist = pd.DataFrame(peak_pos_dist)
df_peak_pos_dist = df_peak_pos_dist.reset_index()
print(f"Mean Highest Ranking: {df_cleaned_genre['Max_Peak_Position'].mean():.0f}")
print(f"Standard Deviation of Highest Ranking: {df_cleaned_genre['Max_Peak_Position'].std():.1f}")

In [ ]:
plt.bar(df_peak_pos_dist['Max_Peak_Position'], df_peak_pos_dist['count'], color='skyblue')
plt.xlabel('Highest Ranking')
plt.ylabel('Number of Songs')
plt.title('Frequency of Peak Rankings')

In [ ]:
max_rank_change = df_cleaned_genre['Max_Rank_Change'].value_counts()
df_max_rank_change = pd.DataFrame(max_rank_change)
df_max_rank_change = df_max_rank_change.reset_index()
df_max_rank_change = df_max_rank_change[df_max_rank_change['Max_Rank_Change'] > 0]
print(f"Mean Largest Week over Week Rank Change: {df_cleaned_genre['Max_Rank_Change'].mean():.0f}")
print(f"Standard Deviation of Largest Week over Week Rank Change: {df_cleaned_genre['Max_Rank_Change'].std():.1f}")

In [ ]:
plt.bar(df_max_rank_change['Max_Rank_Change'], df_max_rank_change['count'], color='orange')
plt.xlabel('Largest Week over Week Rank Change')
plt.ylabel('Number of Songs')
plt.title('Frequency of Largest Week over Week Rank Change')


## Data Preparation

### Initial Data Selection

In [5]:
# removing hotlist df attributes that will not be used in cleaning or analysis
df_hotlist_all = df_hotlist_all.drop(['index', 'url', 'Song', 'Performer', 'Instance'], axis=1)
# converting WeekID to datetime
df_hotlist_all['WeekID'] = pd.to_datetime(df_hotlist_all['WeekID'], errors='coerce')
df_hotlist_all = df_hotlist_all.sort_values(by='WeekID')

# creating a new hotlist df with only complete year data from 2000 - 2020, the time period being studied
df_hotlist_2000s = df_hotlist_all.loc[(df_hotlist_all['WeekID'] > '1999-12-31') & (df_hotlist_all['WeekID'] < '2021-01-01')]

# adding a column to calculate the week over week change in rank
def diff(a, b):
    return a - b

df_hotlist_2000s['Rank_Change'] = df_hotlist_2000s.apply(lambda x: diff(x['Week Position'], x['Previous Week Position']), axis=1)
# replacing NaNs with 0
df_hotlist_2000s['Rank_Change'] = df_hotlist_2000s['Rank_Change'].fillna(0)

# removing features df attributes that will not be used in cleaning or analysis
df_features_all = df_features_all.drop(['index', 'Performer', 'Song', 'spotify_track_album', 
                                        'spotify_track_id', 'spotify_track_preview_url',  
                                        'spotify_track_explicit', 'spotify_track_popularity'], axis=1)


C:\Users\markh\AppData\Local\Temp\ipykernel_9796\3177740419.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hotlist_2000s['Rank_Change'] = df_hotlist_2000s.apply(lambda x: diff(x['Week Position'], x['Previous Week Position']), axis=1)
C:\Users\markh\AppData\Local\Temp\ipykernel_9796\3177740419.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hotlist_2000s['Rank_Change'] = df_hotlist_2000s['Rank_Change'].fillna(0)


In [6]:
# combining the hotlist and features into one dataframe

df_hotlist_and_features_2000s = pd.merge(df_hotlist_2000s, df_features_all, on='SongID', how='left')
df_hotlist_and_features_2000s.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112098 entries, 0 to 112097
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   WeekID                     112098 non-null  datetime64[ns]
 1   Week Position              112098 non-null  int64         
 2   SongID                     112098 non-null  object        
 3   Previous Week Position     101571 non-null  float64       
 4   Peak Position              112098 non-null  int64         
 5   Weeks on Chart             112098 non-null  int64         
 6   Rank_Change                112098 non-null  float64       
 7   spotify_genre              108091 non-null  object        
 8   spotify_track_duration_ms  104590 non-null  float64       
 9   danceability               104287 non-null  float64       
 10  energy                     104287 non-null  float64       
 11  key                        104287 non-null  float64 

### Feature Engineering

In [7]:
"""
The dataset has genre in a single column and the entry for each song has a variety of genres listed in that single column. 
This does not allow me to explore genre in a systematic way.
I'll need to break genre out so that each genre has its own column with a 1 or 0 to indicate whether each song is tagged with that genre (ending with a one-hot encoded structure).
"""

# generating a df with unique genre names
unique_genres = list(set(
    genre 
    for genre_string in df_hotlist_and_features_2000s['spotify_genre'] 
    if pd.notna(genre_string)
    for genre in ast.literal_eval(genre_string)
))

df_unique_genres = pd.DataFrame(unique_genres, columns=['genre'])

# adding counts of each unique genre name
# Extract all genres (with duplicates) and count them
all_genres_list = []
for genre_string in df_hotlist_and_features_2000s['spotify_genre']:
    if pd.notna(genre_string):
        genre_list = ast.literal_eval(genre_string)
        all_genres_list.extend(genre_list)

# Count occurrences
genre_counts = Counter(all_genres_list)

# Map counts to genres dataframe
df_unique_genres['count'] = df_unique_genres['genre'].map(genre_counts)
df_unique_genres = df_unique_genres.sort_values('count', ascending=False)

In [ ]:
# writing to csv for easier review of the data
df_unique_genres.to_csv('genre_counts.csv', index=False)

After reviewing the full set of genre counts, I'm only including genres that appear in 100 or more songs (i.e. at least 0.1% of songs).

In [8]:
# loading list of genres with 100 or more instances in df_cleaned
df_genres_100_up = pd.read_csv('genre_counts_100+inst.csv')

# converting df to list
final_genres_list = df_genres_100_up['genre'].tolist()

# manually one-hot encoding each genre

# creating a list of genres and counts
genre_data = []

for genre_string in df_hotlist_and_features_2000s['spotify_genre'] :
    if pd.notna(genre_string):
        genre_list = ast.literal_eval(genre_string)
        row_dict = {genre: (1 if genre in genre_list else 0) for genre in final_genres_list} # dict with 1 if genre exists in list, 0 if not
    else:
         row_dict = {genre: 0 for genre in final_genres_list} # 0 of genre does not exist in list
    genre_data.append(row_dict)

# creating a df with the list of dicts
genre_df = pd.DataFrame(genre_data)

# concatenating genre data into df_clean
df_hotlist_and_features_2000s = pd.concat([df_hotlist_and_features_2000s, genre_df], axis=1)


In [9]:
# removing spotify_genre and spot-checking resulting df 
pd.set_option('display.max_columns', None)
df_hotlist_and_features_2000s = df_hotlist_and_features_2000s.drop(['spotify_genre'], axis=1)
df_hotlist_and_features_2000s.head()

,WeekID,Week Position,SongID,Previous Week Position,Peak Position,Weeks on Chart,Rank_Change,spotify_track_duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,pop,dance pop,pop rap,rap,contemporary country,country,country road,post-teen pop,hip hop,r&b,urban contemporary,trap,southern hip hop,pop rock,hip pop,modern country rock,neo mellow,post-grunge,gangster rap,atl hip hop,alternative metal,neo soul,dirty south rap,country dawn,modern rock,nu metal,canadian pop,rock,melodic rap,deep pop r&b,edm,new jack swing,permanent wave,miami hip hop,country pop,oklahoma country,latin,tropical house,electropop,uk pop,east coast hip hop,alternative rock,viral pop,quiet storm,chicago rap,redneck,pop punk,crunk,country rock,toronto rap,canadian hip hop,boy band,hardcore hip hop,queens hip hop,talent show,emo,europop,acoustic pop,alternative r&b,conscious hip hop,australian pop,detroit hip hop,rap rock,latin pop,barbadian pop,g funk,girl group,canadian rock,tropical,piano rock,indie pop,electro house,indie poptimism,rap metal,west coast rap,new orleans rap,metropopolis,candy pop,lilith,australian country,philly rap,funk metal,reggaeton,dfw rap,canadian contemporary r&b,soul,mexican pop,adult standards,nc hip hop,british soul,trap queen,hollywood,arkansas country,atl trap,underground hip hop,texas country,uk dance,house,new wave pop,brostep,dancehall,progressive house,funk,singer-songwriter,latin hip hop,idol,garage rock,mellow gold,baroque pop,big room,art pop,reggae fusion,cali rap,bronx hip hop,folk-pop,country rap,stomp and holler,neon pop punk,emo rap,punk,indie rock,funk rock,memphis hip hop,modern alternative rock,lgbtq+ hip hop,progressive electro house,alternative hip hop,blues rock,colombian pop,eurodance,classic rock,baton rouge rap,australian dance,folk,pop emo,soft rock,motown,pixie,canadian country,wrestling,glee club,complextro,vapor trap,etherpop,pittsburgh rap,escape room,indietronica,comic,german techno,new jersey rap,trap latino,houston rap,social media pop,puerto rican pop,deep southern trap,heartland rock,alternative dance,bubblegum dance,alberta country,outlaw country,country gospel,florida rap,hard rock,canadian metal,christian rock,soca,indiecoustica,harlem hip hop,new rave,electronic trap,christian music,grunge,show tunes,viral trap,la indie,swedish pop,swedish electropop,reggaeton flow,dance-punk,celtic rock,socal pop punk,lounge,chicano rap,stomp pop,ccm,vocal jazz,glam metal,worship,irish rock,electropowerpop,electro,indie pop rap,canadian contemporary country,bounce,christian alternative rock,south african rock,deep talent show,disco,hyphy,disco house,canadian latin,australian hip hop,nyc rap,brill building pop,k-pop,nz pop,minnesota hip hop,modern blues rock,album rock,modern folk rock,uk americana,old school hip hop,punk blues,dmv rap,industrial metal,skate punk,swedish synthpop,moombahton
0,2000-01-01,69,Deck The HallsSHeDAISY,97.0,69,2,-28.0,229773.0,0.575,0.837,1.0,-7.141,0.0,0.0406,0.0195,0.000009,0.144,0.444,118.827,4.0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2000-01-01,83,Guerrilla RadioRage Against The Machine,87.0,69,10,-4.0,206200.0,0.599,0.957,11.0,-5.764,1.0,0.1880,0.0129,0.000071,0.155,0.489,103.680,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [10]:
# creating new df with most 3 most popular genres by week

# Get all genre column names
genre_start_idx = df_hotlist_and_features_2000s.columns.get_loc('pop')
genre_cols = df_hotlist_and_features_2000s.columns[genre_start_idx:].tolist()

# Group by WeekID and sum the genre columns to get counts
genre_counts = df_hotlist_and_features_2000s.groupby('WeekID')[genre_cols].sum()

# For each week, find the top 3 genres
top_genres = []
for week_id in genre_counts.index:
    # Get the genre counts for this week and sort them
    week_genres = genre_counts.loc[week_id].sort_values(ascending=False)
    
    # Get the top 3 genre names
    top_3 = week_genres.head(3).index.tolist()
    
    # Pad with None if there are fewer than 3 genres
    while len(top_3) < 3:
        top_3.append(None)
    
    top_genres.append({
        'WeekID': week_id,
        'Most_Popular_Genre': top_3[0],
        '2nd_Most_Popular_Genre': top_3[1],
        '3rd_Most_Popular_Genre': top_3[2]
    })

# Create the new dataframe
df_top_genres = pd.DataFrame(top_genres)


In [34]:
df_top_genres

,WeekID,Most_Popular_Genre,2nd_Most_Popular_Genre,3rd_Most_Popular_Genre
0,2000-01-01,dance pop,r&b,urban contemporary
1,2000-01-08,dance pop,r&b,urban contemporary
2,2000-01-15,dance pop,r&b,urban contemporary
3,2000-01-22,dance pop,r&b,hip hop
4,2000-01-29,dance pop,r&b,urban contemporary
...,...,...,...,...
1091,2020-11-28,pop,contemporary country,dance pop
1092,2020-12-05,pop,contemporary country,dance pop
1093,2020-12-12,adult standards,pop,contemporary country
1094,2020-12-19,adult standards,pop,contemporary country


In [35]:
# adding a column to the main df indicating whether each song is in a genre that's in the top 3 genres for a given week

def is_in_top3_genres(row, df_top_genres):
    week = row['WeekID']

    top_genres = df_top_genres[df_top_genres['WeekID'] == week]

    if len(top_genres) == 0:
        return 0
    
    top_3 = [
        top_genres.iloc[0]['Most_Popular_Genre'],
        top_genres.iloc[0]['2nd_Most_Popular_Genre'],
        top_genres.iloc[0]['3rd_Most_Popular_Genre']
        ]

    for genre in top_3:
        if genre in row.index and row[genre] == 1:
            return 1
        
        else:
            return 0
        
df_hotlist_and_features_2000s['in_top3_genres'] = df_hotlist_and_features_2000s.apply(
    lambda row: is_in_top3_genres(row, df_top_genres), axis=1
)

In [37]:
# new df with the max weekly rank change for each song 
df_max_rank_change = df_hotlist_and_features_2000s.groupby('SongID', as_index=False)['Rank_Change'].max()
df_max_rank_change.rename(columns={'Rank_Change': 'Max_Rank_Change'}, inplace=True)
df_max_rank_change.set_index('SongID', inplace=True)

# new df with the max peak rank for each song 
df_max_peak_pos = df_hotlist_and_features_2000s.groupby('SongID', as_index=False)['Peak Position'].max()
df_max_peak_pos.rename(columns={'Peak Position': 'Max_Peak_Position'}, inplace=True)
df_max_peak_pos.set_index('SongID', inplace=True)

# ensuring these new dfs have no null values
df_max_rank_change['Max_Rank_Change'].isna().sum(), df_max_peak_pos['Max_Peak_Position'].isna().sum()

(0, 0)

In [ ]:
# removing duplicates and rechecking
df_features_2000s = df_features_2000s.drop_duplicates(subset='SongID')

print(len(df_features_2000s))
print(len(pd.unique(df_features_2000s['SongID'])))

In [38]:
# adding max peak position to main df
df_2000s_data = df_hotlist_and_features_2000s.join(df_max_peak_pos, on='SongID')

# adding max rank change to features df
df_2000s_data = df_2000s_data.join(df_max_rank_change, on='SongID')

df_2000s_data.head()

,WeekID,Week Position,SongID,Previous Week Position,Peak Position,Weeks on Chart,Rank_Change,spotify_track_duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,pop,dance pop,pop rap,rap,contemporary country,country,country road,post-teen pop,hip hop,r&b,urban contemporary,trap,southern hip hop,pop rock,hip pop,modern country rock,neo mellow,post-grunge,gangster rap,atl hip hop,alternative metal,neo soul,dirty south rap,country dawn,modern rock,nu metal,canadian pop,rock,melodic rap,deep pop r&b,edm,new jack swing,permanent wave,miami hip hop,country pop,oklahoma country,latin,tropical house,electropop,uk pop,east coast hip hop,alternative rock,viral pop,quiet storm,chicago rap,redneck,pop punk,crunk,country rock,toronto rap,canadian hip hop,boy band,hardcore hip hop,queens hip hop,talent show,emo,europop,acoustic pop,alternative r&b,conscious hip hop,australian pop,detroit hip hop,rap rock,latin pop,barbadian pop,g funk,girl group,canadian rock,tropical,piano rock,indie pop,electro house,indie poptimism,rap metal,west coast rap,new orleans rap,metropopolis,candy pop,lilith,australian country,philly rap,funk metal,reggaeton,dfw rap,canadian contemporary r&b,soul,mexican pop,adult standards,nc hip hop,british soul,trap queen,hollywood,arkansas country,atl trap,underground hip hop,texas country,uk dance,house,new wave pop,brostep,dancehall,progressive house,funk,singer-songwriter,latin hip hop,idol,garage rock,mellow gold,baroque pop,big room,art pop,reggae fusion,cali rap,bronx hip hop,folk-pop,country rap,stomp and holler,neon pop punk,emo rap,punk,indie rock,funk rock,memphis hip hop,modern alternative rock,lgbtq+ hip hop,progressive electro house,alternative hip hop,blues rock,colombian pop,eurodance,classic rock,baton rouge rap,australian dance,folk,pop emo,soft rock,motown,pixie,canadian country,wrestling,glee club,complextro,vapor trap,etherpop,pittsburgh rap,escape room,indietronica,comic,german techno,new jersey rap,trap latino,houston rap,social media pop,puerto rican pop,deep southern trap,heartland rock,alternative dance,bubblegum dance,alberta country,outlaw country,country gospel,florida rap,hard rock,canadian metal,christian rock,soca,indiecoustica,harlem hip hop,new rave,electronic trap,christian music,grunge,show tunes,viral trap,la indie,swedish pop,swedish electropop,reggaeton flow,dance-punk,celtic rock,socal pop punk,lounge,chicano rap,stomp pop,ccm,vocal jazz,glam metal,worship,irish rock,electropowerpop,electro,indie pop rap,canadian contemporary country,bounce,christian alternative rock,south african rock,deep talent show,disco,hyphy,disco house,canadian latin,australian hip hop,nyc rap,brill building pop,k-pop,nz pop,minnesota hip hop,modern blues rock,album rock,modern folk rock,uk americana,old school hip hop,punk blues,dmv rap,industrial metal,skate punk,swedish synthpop,moombahton,in_top3_genres,Max_Peak_Position,Max_Rank_Change
0,2000-01-01,69,Deck The HallsSHeDAISY,97.0,69,2,-28.0,229773.0,0.575,0.837,1.0,-7.141,0.0,0.0406,0.0195,0.000009,0.144,0.444,118.827,4.0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,69,-8.0
1,2000-01-01,83,Guerrilla RadioRage Against The Machine,87.0,69,10,-4.0,206200.0,0.599,0.957,11.0,-5.764,1.0,0.1880,0.0129,0.000071,0.155,0.489,103.680,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [26]:
all_cols = [(index, column) for index, column in enumerate(df_2000s_data.columns)]
print(all_cols)

[(0, 'WeekID'), (1, 'Week Position'), (2, 'SongID'), (3, 'Previous Week Position'), (4, 'Peak Position'), (5, 'Weeks on Chart'), (6, 'Rank_Change'), (7, 'spotify_track_duration_ms'), (8, 'danceability'), (9, 'energy'), (10, 'key'), (11, 'loudness'), (12, 'mode'), (13, 'speechiness'), (14, 'acousticness'), (15, 'instrumentalness'), (16, 'liveness'), (17, 'valence'), (18, 'tempo'), (19, 'time_signature'), (20, 'pop'), (21, 'dance pop'), (22, 'pop rap'), (23, 'rap'), (24, 'contemporary country'), (25, 'country'), (26, 'country road'), (27, 'post-teen pop'), (28, 'hip hop'), (29, 'r&b'), (30, 'urban contemporary'), (31, 'trap'), (32, 'southern hip hop'), (33, 'pop rock'), (34, 'hip pop'), (35, 'modern country rock'), (36, 'neo mellow'), (37, 'post-grunge'), (38, 'gangster rap'), (39, 'atl hip hop'), (40, 'alternative metal'), (41, 'neo soul'), (42, 'dirty south rap'), (43, 'country dawn'), (44, 'modern rock'), (45, 'nu metal'), (46, 'canadian pop'), (47, 'rock'), (48, 'melodic rap'), (49, 

In [30]:
"""
I'd like to see how similar each song is to the #1 song on a week by week basis. 
I'll do this by doing a similarity analysis using Manhattan distance, since the data is both sparse and high-dimensional.
"""
attribute_columns = df_2000s_data.columns[7:238].tolist()

scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_2000s_data[attribute_columns])

def calc_weekly_manhattan(week_df):
    # attributes of #1 song
    top_song = week_df[week_df['Week Position'] == 1][attribute_columns].values[0]

    # calculating distance
    distances = pairwise_distances(
        week_df[attribute_columns].values,
        top_song.reshape(1, -1),
        metric='manhattan'
    ).flatten() 

    return distances

df_2000s_data['manhattan_dist'] = df_2000s_data.groupby('WeekID', group_keys=False).apply(
                lambda x: pd.Series(calc_weekly_manhattan(x), index=x.index)
)

# converting distance to similarity score   
df_2000s_data['similarity'] = 1 / (1 + df_2000s_data['manhattan_dist'])


ValueError: Input contains NaN.

In [ ]:
df_songID_genre = df_cleaned.drop(['spotify_track_id', 'spotify_track_duration_ms', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
                                   'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'Max_Peak_Position', 'Max_Rank_Change'], axis=1)
df_songID_genre.head(2)

I created two datasets: one containing genre and one without. This will allow me to model this data with and without genre.

In [ ]:
# removing fields used for prep/cleaning but not needed for analysis
df_cleaned = df_cleaned.drop(['SongID', 'spotify_genre', 'spotify_track_id'], axis=1)

# my code added columns for all genres in spotify_genre, removing unwanted columns and creating a clean df with genre
last_col_to_keep_genre = 'emo rap'
df_cleaned_genre = df_cleaned.loc[:, :last_col_to_keep_genre]
# removing NaN rows
df_cleaned_genre = df_cleaned_genre.dropna()

# creating a clean df for analysis without genre
last_col_to_keep_no_genre = 'Max_Rank_Change'
df_cleaned_no_genre = df_cleaned.loc[:, :last_col_to_keep_no_genre]
# removing NaN rows
df_cleaned_genre = df_cleaned_genre.dropna()
df_cleaned_no_genre = df_cleaned_no_genre.dropna()

df_cleaned_genre.info(), df_cleaned_no_genre.info()

### Features and Target Variables

I'm prepping 4 versions for XGBoost and k-NN:

1. Max Peak Position, no genre (1__1 variables)
2. Max Peak Position, with genre (1_2 variables)
3. Max Rank Change, no genre (2_1 variables)
4. Max Rank Change, with genre (2_2 variables)

In [ ]:
# Prepare features and target for XGBoost and k-NN max_peak_position analysis, no genre
X1_1 = df_cleaned_no_genre.drop(['Max_Peak_Position'], axis=1)
y1_1 = df_cleaned_no_genre['Max_Peak_Position']

# Splitting the data into training and testing sets (75-25 split and random_state of 42)
X1_1_train, X1_1_test, y1_1_train, y1_1_test = train_test_split(X1_1, y1_1, test_size=0.25, random_state=42)

# Standardize the features
scaler = StandardScaler()
X1_1_train_scaled = scaler.fit_transform(X1_1_train)
X1_1_test_scaled = scaler.fit_transform(X1_1_test)

In [ ]:
# Prepare features and target for XGBoost and k-NN max_peak_position analysis, including genre
X1_2 = df_cleaned_genre.drop(['Max_Peak_Position'], axis=1)
y1_2 = df_cleaned_genre['Max_Peak_Position']

# Splitting the data into training and testing sets (75-25 split and random_state of 42)
X1_2_train, X1_2_test, y1_2_train, y1_2_test = train_test_split(X1_2, y1_2, test_size=0.25, random_state=42)

# Standardize the features
scaler = StandardScaler()
X1_2_train_scaled = scaler.fit_transform(X1_2_train)
X1_2_test_scaled = scaler.fit_transform(X1_2_test)

In [ ]:
# Prepare features and target for XGBoost and k-NN max_rank_change analysis, no genre
X2_1 = df_cleaned_no_genre.drop(['Max_Peak_Position', 'Max_Rank_Change'], axis=1)
y2_1 = df_cleaned_no_genre['Max_Rank_Change']

# Splitting the data into training and testing sets (75-25 split and random_state of 42)
X2_1_train, X2_1_test, y2_1_train, y2_1_test = train_test_split(X2_1, y2_1, test_size=0.25, random_state=42)

# Standardize the features
scaler = StandardScaler()
X2_1_train_scaled = scaler.fit_transform(X2_1_train)
X2_1_test_scaled = scaler.fit_transform(X2_1_test)

In [ ]:
# Prepare features and target for XGBoost and k-NN max_rank_change analysis, including genre
X2_2 = df_cleaned_genre.drop(['Max_Peak_Position', 'Max_Rank_Change'], axis=1)
y2_2 = df_cleaned_genre['Max_Rank_Change']

# Splitting the data into training and testing sets (75-25 split and random_state of 42)
X2_2_train, X2_2_test, y2_2_train, y2_2_test = train_test_split(X2_2, y2_2, test_size=0.25, random_state=42)

# Standardize the features
scaler = StandardScaler()
X2_2_train_scaled = scaler.fit_transform(X2_2_train)
X2_2_test_scaled = scaler.fit_transform(X2_2_test)

Another 4 versions of the data the deep learning model

1. Max Peak Position, no genre (3__1 variables)
2. Max Peak Position, with genre (3_2 variables)
3. Max Rank Change, no genre (4_1 variables)
4. Max Rank Change, with genre (4_2 variables)

In [ ]:
# features and target for deep learning max_peak_position analysis, no genre
X3_1 = df_cleaned_no_genre.drop(['Max_Peak_Position'], axis=1)
y3_1 = df_cleaned_no_genre['Max_Peak_Position']

# Splitting the data into training and testing sets 
X3_1_train, X3_1_test, y3_1_train, y3_1_test = train_test_split(X3_1, y3_1, test_size=0.2, random_state=42)

# splitting training data into training and validiation
X3_1_train_final, X3_1_val, y3_1_train_final, y3_1_val = train_test_split(X3_1_train, y3_1_train, test_size=0.2, random_state=42)

# normalizing 
scaler.fit(X3_1_train_final)
X3_1_train_scaled = scaler.transform(X3_1_train_final)
X3_1_val_scaled = scaler.transform(X3_1_val)
X3_1_test_scaled = scaler.transform(X3_1_test)

In [ ]:
# features and target for deep learning max_peak_position analysis, including genre
X3_2 = df_cleaned_genre.drop(['Max_Peak_Position'], axis=1)
y3_2 = df_cleaned_genre['Max_Peak_Position']

# Splitting the data into training and testing sets 
X3_2_train, X3_2_test, y3_2_train, y3_2_test = train_test_split(X3_2, y3_2, test_size=0.2, random_state=42)

# splitting training data into training and validiation
X3_2_train_final, X3_2_val, y3_2_train_final, y3_2_val = train_test_split(X3_2_train, y3_2_train, test_size=0.2, random_state=42)

# normalizing
scaler.fit(X3_2_train_final)
X3_2_train_scaled = scaler.transform(X3_2_train_final)
X3_2_val_scaled = scaler.transform(X3_2_val)
X3_2_test_scaled = scaler.transform(X3_2_test)

In [ ]:
# features and target for deep learning max_rank_change analysis, no genre
X4_1 = df_cleaned_no_genre.drop(['Max_Peak_Position', 'Max_Rank_Change'], axis=1)
y4_1 = df_cleaned_no_genre['Max_Rank_Change']

# Splitting the data into training and testing sets 
X4_1_train, X4_1_test, y4_1_train, y4_1_test = train_test_split(X4_1, y4_1, test_size=0.2, random_state=42)

# splitting training data into training and validiation
X4_1_train_final, X4_1_val, y4_1_train_final, y4_1_val = train_test_split(X4_1_train, y4_1_train, test_size=0.2, random_state=42)

# normalizing
scaler.fit(X4_1_train_final)
X4_1_train_scaled = scaler.transform(X4_1_train_final)
X4_1_val_scaled = scaler.transform(X4_1_val)
X4_1_test_scaled = scaler.transform(X4_1_test)

In [ ]:
# Prepare features and target for simple deep learning max_rank_change analysis, including genre
X4_2 = df_cleaned_genre.drop(['Max_Peak_Position', 'Max_Rank_Change'], axis=1)
y4_2 = df_cleaned_genre['Max_Rank_Change']

# Splitting the data into training and testing sets 
X4_2_train, X4_2_test, y4_2_train, y4_2_test = train_test_split(X4_2, y4_2, test_size=0.2, random_state=42)

# splitting training data into training and validiation
X4_2_train_final, X4_2_val, y4_2_train_final, y4_2_val = train_test_split(X4_2_train, y4_2_train, test_size=0.2, random_state=42)

# normalizing
scaler.fit(X4_2_train_final)
X4_2_train_scaled = scaler.transform(X4_2_train_final)
X4_2_val_scaled = scaler.transform(X4_2_val)
X4_2_test_scaled = scaler.transform(X4_2_test)

## Analysis

### XGBoost

**XGBoost | Max Peak Position - No Genre**

In [ ]:
# XGBoost for max_peak_position, no genre

xgb_model1_1 = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)

xgb_model1_1.fit(X1_1_train, y1_1_train)
y1_1_pred = xgb_model1_1.predict(X1_1_test)
y1_1_pred = np.clip(np.round(y1_1_pred), 1, 100)

rmse1_1 = np.sqrt(mean_squared_error(y1_1_test, y1_1_pred))
r2_1_1 = r2_score(y1_1_test, y1_1_pred)

print(f'RMSE: {rmse1_1:.3f}')
print(f'R²: {r2_1_1:.3f}')

In [ ]:
# hyperparameter tuning 1

param_grid1 = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

grid_search_xgb1_1 = GridSearchCV(estimator=xgb_model1_1,
                            param_grid=param_grid1,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb1_1.fit(X1_1_train, y1_1_train)

print("Best parameters:", grid_search_xgb1_1.best_params_)

In [ ]:
# hyperparameter tuning 2

param_grid2 = {
    'max_depth': [2, 3, 4],
    'learning_rate': [0.05, 0.1, 0.15,],
    'subsample': [0.75, 0.8, 0.85],
    'colsample_bytree': [0.7, 0.8, 0.9],
}

grid_search_xgb1_1 = GridSearchCV(estimator=xgb_model1_1,
                            param_grid=param_grid2,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb1_1.fit(X1_1_train, y1_1_train)

print("Best parameters:", grid_search_xgb1_1.best_params_)

In [ ]:
# hyperparameter tuning 3

param_grid3 = {
    'max_depth': [4, 5],
    'learning_rate': [0.03, 0.05, 0.07],
    'subsample': [0.75],
    'colsample_bytree': [0.8],
}

grid_search_xgb1_1 = GridSearchCV(estimator=xgb_model1_1,
                            param_grid=param_grid3,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb1_1.fit(X1_1_train, y1_1_train)

print("Best parameters:", grid_search_xgb1_1.best_params_)

In [ ]:
# Extract best/final model for max_peak_position
best_xgb1_1 = grid_search_xgb1_1.best_estimator_

# predictions
y1_1_pred_best = best_xgb1_1.predict(X1_1_test)
y1_1_pred_best = np.clip(np.round(y1_1_pred_best), 1, 100)

# Evaluate XGBoost model
rmse1_1_best = np.sqrt(mean_squared_error(y1_1_test, y1_1_pred_best))
r2_1_1_best = r2_score(y1_1_test, y1_1_pred_best)

print(f'RMSE: {rmse1_1_best:.3f}')
print(f'R²: {r2_1_1_best:.3f}')

**XGBoost | Max Peak Position - With Genre**

In [ ]:
# XGBoost for max_peak_position, with genre

xgb_model1_2 = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)

xgb_model1_2.fit(X1_2_train, y1_2_train)
y1_2_pred = xgb_model1_2.predict(X1_2_test)
y1_2_pred = np.clip(np.round(y1_2_pred), 1, 100)

rmse1_2 = np.sqrt(mean_squared_error(y1_2_test, y1_2_pred))
r2_1_2 = r2_score(y1_2_test, y1_2_pred)

print(f'RMSE: {rmse1_2:.3f}')
print(f'R²: {r2_1_2:.3f}')

In [ ]:
# hyperparameter tuning 1

param_grid1 = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

grid_search_xgb1_2 = GridSearchCV(estimator=xgb_model1_2,
                            param_grid=param_grid1,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb1_2.fit(X1_2_train, y1_2_train)

print("Best parameters:", grid_search_xgb1_2.best_params_)

In [ ]:
# hyperparameter tuning 2

param_grid4 = {
    'max_depth': [2, 3, 4],
    'learning_rate': [0.05, 0.1, 0.15],
    'subsample': [0.9, 1.0, 1.1],
    'colsample_bytree': [0.6, 0.7, 0.8],
}

grid_search_xgb1_2 = GridSearchCV(estimator=xgb_model1_2,
                            param_grid=param_grid4,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb1_2.fit(X1_2_train, y1_2_train)

print("Best parameters:", grid_search_xgb1_2.best_params_)

In [ ]:
# hyperparameter tuning 3

param_grid5 = {
    'max_depth': [4, 5],
    'learning_rate': [0.03, 0.05, 0.07],
    'subsample': [0.9],
    'colsample_bytree': [0.8],
}

grid_search_xgb1_2 = GridSearchCV(estimator=xgb_model1_2,
                            param_grid=param_grid5,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb1_2.fit(X1_2_train, y1_2_train)

print("Best parameters:", grid_search_xgb1_2.best_params_)

In [ ]:
# Extract best/final model 
best_xgb1_2 = grid_search_xgb1_2.best_estimator_

# predictions
y1_2_pred_best = best_xgb1_2.predict(X1_2_test)
y1_2_pred_best = np.clip(np.round(y1_2_pred_best), 1, 100)

# Evaluate XGBoost model
rmse1_2_best = np.sqrt(mean_squared_error(y1_2_test, y1_2_pred_best))
r2_1_2_best = r2_score(y1_2_test, y1_2_pred_best)

print(f'RMSE: {rmse1_2_best:.3f}')
print(f'R²: {r2_1_2_best:.3f}')

**XGBoost | Max Rank Change - No Genre**

In [ ]:
# XGBoost for max_rank_change

xgb_model2_1 = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)

xgb_model2_1.fit(X2_1_train, y2_1_train)
y2_1_pred = xgb_model2_1.predict(X2_1_test)
y2_1_pred = np.clip(np.round(y2_1_pred), 1, 100)

rmse2_1 = np.sqrt(mean_squared_error(y2_1_test, y2_1_pred))
r2_2_1 = r2_score(y2_1_test, y2_1_pred)

print(f'RMSE: {rmse2_1:.3f}')
print(f'R²: {r2_2_1:.3f}')

In [ ]:
# hyperparameter tuning 1 for max_rank_change

param_grid1 = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

grid_search_xgb2_1 = GridSearchCV(estimator=xgb_model2_1,
                            param_grid=param_grid1,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb2_1.fit(X2_1_train, y2_1_train)

print("Best parameters:", grid_search_xgb2_1.best_params_)

In [ ]:
# hyperparameter tuning 2 for max_rank_change

param_grid4 = {
    'max_depth': [2, 3, 4],
    'learning_rate': [0.005, 0.01, 0.015],
    'subsample': [1.0, 1.1, 1.2],
    'colsample_bytree': [0.7, 0.8, 0.9],
}

grid_search_xgb2_1 = GridSearchCV(estimator=xgb_model2_1,
                            param_grid=param_grid4,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb2_1.fit(X2_1_train, y2_1_train)

print("Best parameters:", grid_search_xgb2_1.best_params_)

In [ ]:
# hyperparameter tuning 3 for max_rank_change

param_grid5 = {
    'max_depth': [3],
    'learning_rate': [0.003, 0.005, 0.007],
    'subsample': [1.0],
    'colsample_bytree': [0.9],
        }

grid_search_xgb2_1 = GridSearchCV(estimator=xgb_model2_1,
                            param_grid=param_grid5,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb2_1.fit(X2_1_train, y2_1_train)

print("Best parameters:", grid_search_xgb2_1.best_params_)

In [ ]:
# Extract best/final model  for max_rank_change
best_xgb2_1 = grid_search_xgb2_1.best_estimator_

y2_1_pred_best = best_xgb2_1.predict(X2_1_test)
y2_1_pred_best = np.clip(np.round(y2_1_pred_best), 1, 100)

# Evaluate XGBoost model
rmse2_1_best = np.sqrt(mean_squared_error(y2_1_test, y2_1_pred_best))
r2_2_1_best = r2_score(y2_1_test, y2_1_pred_best)

print(f'RMSE: {rmse2_1_best:.3f}')
print(f'R²: {r2_2_1_best:.3f}')

**XGBoost | Max Rank Change - With Genre**

In [ ]:
# XGBoost for max_rank_change

xgb_model2_2 = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)

xgb_model2_2.fit(X2_2_train, y2_2_train)
y2_2_pred = xgb_model2_2.predict(X2_2_test)
y2_2_pred = np.clip(np.round(y2_2_pred), 1, 100)

rmse2_2 = np.sqrt(mean_squared_error(y2_2_test, y2_2_pred))
r2_2_2 = r2_score(y2_2_test, y2_2_pred)

print(f'RMSE: {rmse2_2:.3f}')
print(f'R²: {r2_2_2:.3f}')

In [ ]:
# hyperparameter tuning 1

param_grid1 = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

grid_search_xgb2_2 = GridSearchCV(estimator=xgb_model2_2,
                            param_grid=param_grid1,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb2_2.fit(X2_2_train, y2_2_train)

print("Best parameters:", grid_search_xgb2_2.best_params_)

In [ ]:
# hyperparameter tuning 2

param_grid6 = {
    'max_depth': [2, 3, 4],
    'learning_rate': [0.005, 0.01, 0.015],
    'subsample': [1.0, 1.1, 1.2],
    'colsample_bytree': [0.6, 0.7, 0.8],
}

grid_search_xgb2_2 = GridSearchCV(estimator=xgb_model2_2,
                            param_grid=param_grid6,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb2_2.fit(X2_2_train, y2_2_train)

print("Best parameters:", grid_search_xgb2_2.best_params_)

In [ ]:
# hyperparameter tuning 3

param_grid7 = {
    'max_depth': [3],
    'learning_rate': [0.003, 0.005, 0.007],
    'subsample': [1.0],
    'colsample_bytree': [0.8],
}

grid_search_xgb2_2 = GridSearchCV(estimator=xgb_model2_2,
                            param_grid=param_grid7,
                            cv=5,
                            n_jobs=-1,
                            verbose=1)
grid_search_xgb2_2.fit(X2_2_train, y2_2_train)

print("Best parameters:", grid_search_xgb2_2.best_params_)

In [ ]:
# Extract best/final model  for max_rank_change
best_xgb2_2 = grid_search_xgb2_2.best_estimator_

y2_2_pred_best = best_xgb2_2.predict(X2_2_test)
y2_2_pred_best = np.clip(np.round(y2_2_pred_best), 1, 100)

# Evaluate XGBoost model
rmse2_2_best = np.sqrt(mean_squared_error(y2_2_test, y2_2_pred_best))
r2_2_2_best = r2_score(y2_2_test, y2_2_pred_best)

print(f'RMSE: {rmse2_2_best:.3f}')
print(f'R²: {r2_2_2_best:.3f}')

#### XGBoost Summary

Using XGBoost, including the genre features slightly improved model performance. However, the Max Rank Change models both had an r<sup>2</sup> value less than 0.001, essentially indicating no fit of the model to the test data. Max Peak Position performed better, but the highest r<sup>2</sup> value was 0.138 so their predictive value is low.

Given the lack of predictive power in these outcomes, I'm shifting focus to the other two techniques.

### k-Nearest Neighbors

**k-Nearest Neighbors | Max Peak Position - No Genre**

In [ ]:
# Define parameter grid for standard metrics
param_grid8 = {
    'n_neighbors': list(range(1, 31, 2)),  # Odd values to avoid ties
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'chebyshev'],
    'algorithm': ['auto']
}

# Create standard KNN model for grid search
knn = KNeighborsClassifier()

# Perform grid search
print("Starting grid search for standard metrics...")
grid_search1_1 = GridSearchCV(knn, param_grid8, cv=5, scoring='accuracy', n_jobs=-1)
grid_search1_1.fit(X1_1_train_scaled, y1_1_train)

# Get best parameters and score
standard_best_params1_1 = grid_search1_1.best_params_
standard_best_score1_1 = grid_search1_1.best_score_
print(f"Best parameters (standard metrics): {standard_best_params1_1}")
print(f"Best cross-validation accuracy: {standard_best_score1_1:.4f}")


In [ ]:
# final model with best parameters
final_model1_1 = grid_search1_1.best_estimator_

# predictions on test set
y1_1_pred = final_model1_1.predict(X1_1_test_scaled)

# Evaluate the model
accuracy1_1 = accuracy_score(y1_1_test, y1_1_pred)

print(f"Best accuracy: {accuracy1_1:.4f}")

**k-Nearest Neighbors | Max Peak Position - With Genre**

In [ ]:
# arameter grid for standard metrics
param_grid8 = {
    'n_neighbors': list(range(1, 31, 2)),  # Odd values to avoid ties
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'chebyshev'],
    'algorithm': ['auto']
}

# Create standard KNN model for grid search
knn = KNeighborsClassifier()

# Perform grid search
print("Starting grid search for standard metrics...")
grid_search1_2 = GridSearchCV(knn, param_grid8, cv=5, scoring='accuracy', n_jobs=-1)
grid_search1_2.fit(X1_2_train_scaled, y1_2_train)

# Get best parameters and score
standard_best_params1_2 = grid_search1_2.best_params_
standard_best_score1_2 = grid_search1_2.best_score_
print(f"Best parameters (standard metrics): {standard_best_params1_2}")
print(f"Best cross-validation accuracy: {standard_best_score1_2:.4f}")


In [ ]:
# final model with best parameters
final_model1_2 = grid_search1_2.best_estimator_

# predictions on test set
y1_2_pred = final_model1_2.predict(X1_2_test_scaled)

# Evaluate the model
accuracy1_2 = accuracy_score(y1_2_test, y1_2_pred)

print(f"Best accuracy: {accuracy1_2:.4f}")

**k-Nearest Neighbors | Max Rank Change - No Genre**

In [ ]:
# Define parameter grid for standard metrics
param_grid8 = {
    'n_neighbors': list(range(1, 31, 2)),  # Odd values to avoid ties
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'chebyshev'],
    'algorithm': ['auto']
}

# Create standard KNN model for grid search
knn = KNeighborsClassifier()

# Perform grid search
print("Starting grid search for standard metrics...")
grid_search2_1 = GridSearchCV(knn, param_grid8, cv=5, scoring='accuracy', n_jobs=-1)
grid_search2_1.fit(X2_1_train_scaled, y2_1_train)

# Get best parameters and score
standard_best_params2_1 = grid_search2_1.best_params_
standard_best_score2_1 = grid_search2_1.best_score_
print(f"Best parameters (standard metrics): {standard_best_params2_1}")
print(f"Best cross-validation accuracy: {standard_best_score2_1:.4f}")


In [ ]:
# final model with best parameters
final_model2_1 = grid_search2_1.best_estimator_


# Make predictions on test set
y2_1_pred = final_model2_1.predict(X2_1_test_scaled)


# Evaluate the model
accuracy2_1 = accuracy_score(y2_1_test, y2_1_pred)

print(f"Best accuracy: {accuracy2_1:.4f}")

**k-Nearest Neighbors | Max Rank Change - With Genre**

In [ ]:
# Define parameter grid for standard metrics
param_grid8 = {
    'n_neighbors': list(range(1, 31, 2)),  # Odd values to avoid ties
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'chebyshev'],
    'algorithm': ['auto']
}

# Create standard KNN model for grid search
knn = KNeighborsClassifier()

# Perform grid search
print("Starting grid search for standard metrics...")
grid_search2_2 = GridSearchCV(knn, param_grid8, cv=5, scoring='accuracy', n_jobs=-1)
grid_search2_2.fit(X2_2_train_scaled, y2_2_train)

# Get best parameters and score
standard_best_params2_2 = grid_search2_2.best_params_
standard_best_score2_2 = grid_search2_2.best_score_
print(f"Best parameters (standard metrics): {standard_best_params2_2}")
print(f"Best cross-validation accuracy: {standard_best_score2_2:.4f}")


In [ ]:
# final model with best parameters
final_model2_2 = grid_search2_2.best_estimator_


# Make predictions on test set
y2_2_pred = final_model2_2.predict(X2_2_test_scaled)


# Evaluate the model
accuracy2_2 = accuracy_score(y2_2_test, y2_2_pred)

print(f"Best accuracy: {accuracy2_2:.4f}")

#### k-NN Summary

The k-NN models performed poorly on the Max Peak Position data, with a maximum accuracy of 0.052. The performance on the Max Rank Change was better, but the maximum accuracy was still just 0.217.

I will not explore k-NN further and instead focus on the deep learning model.

### Deep Learning

**Deep Learning | Max Peak Position - No Genre**

In [ ]:
# deep learning model, no regularization or dropout

baseline_model3_1 = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X3_1_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Single output for regression
])

baseline_model3_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_baseline3_1 = baseline_model3_1.fit(
    X3_1_train_scaled, y3_1_train_final,
    validation_data=(X3_1_val_scaled, y3_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# deep learning model with batch normalization

bnorm_model3_1 = keras.Sequential([
    layers.Dense(64, activation='linear', input_shape=(X3_1_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),

    layers.Dense(1)  # Single output for regression
])

bnorm_model3_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_bnorm_model3_1 = bnorm_model3_1.fit(
    X3_1_train_scaled, y3_1_train_final,
    validation_data=(X3_1_val_scaled, y3_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# deep learning model regularization (L2 and dropout)

l2_reg = 1e-4
dropout_rate = 0.4

reg_model3_1 = keras.Sequential([
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X3_1_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model3_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model3_1 = reg_model3_1.fit(
    X3_1_train_scaled, y3_1_train_final,
    validation_data=(X3_1_val_scaled, y3_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# model evaluation
print("MODEL EVALUATION: MAX PEAK POSITION, NO GENRE")

print("\n=== Baseline Model ===")
train_scores3_1 = baseline_model3_1.evaluate(X3_1_train_scaled, y3_1_train_final, verbose=0)
val_scores3_1   = baseline_model3_1.evaluate(X3_1_val_scaled, y3_1_val, verbose=0)
print(f"Train MAE: {train_scores3_1[1]:.4f}, Train MSE: {train_scores3_1[2]:.4f}")
print(f"Val   MAE: {val_scores3_1[1]:.4f}, Val   MSE: {val_scores3_1[2]:.4f}")

print("\n=== BatchNorm Model ===")
train_scores_bn3_1 = bnorm_model3_1.evaluate(X3_1_train_scaled, y3_1_train_final, verbose=0)
val_scores_bn3_1   = bnorm_model3_1.evaluate(X3_1_val_scaled, y3_1_val, verbose=0)
print(f"Train MAE: {train_scores_bn3_1[1]:.4f}, Train MSE: {train_scores_bn3_1[2]:.4f}")
print(f"Val   MAE: {val_scores_bn3_1[1]:.4f}, Val   MSE: {val_scores_bn3_1[2]:.4f}")

print("\n=== Regularized Model (L2 + Dropout) ===")
train_scores_reg3_1 = reg_model3_1.evaluate(X3_1_train_scaled, y3_1_train_final, verbose=0)
val_scores_reg3_1   = reg_model3_1.evaluate(X3_1_val_scaled, y3_1_val, verbose=0)
print(f"Train MAE: {train_scores_reg3_1[1]:.4f}, Train MSE: {train_scores_reg3_1[2]:.4f}")
print(f"Val   MAE: {val_scores_reg3_1[1]:.4f}, Val   MSE: {val_scores_reg3_1[2]:.4f}")


**Deep Learning | Max Peak Position - With Genre**

In [ ]:
# deep learning model, no regularization or dropout

baseline_model3_2 = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X3_2_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Single output for regression
])

baseline_model3_2.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_baseline3_2 = baseline_model3_2.fit(
    X3_2_train_scaled, y3_2_train_final,
    validation_data=(X3_2_val_scaled, y3_2_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# deep learning model with batch normalization

bnorm_model3_2 = keras.Sequential([
    layers.Dense(64, activation='linear', input_shape=(X3_2_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),

    layers.Dense(1)  # Single output for regression
])

bnorm_model3_2.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_bnorm_model3_2 = bnorm_model3_2.fit(
    X3_2_train_scaled, y3_2_train_final,
    validation_data=(X3_2_val_scaled, y3_2_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# deep learning model with regularization (L2 and dropout)

l2_reg = 1e-4
dropout_rate = 0.4

reg_model3_2 = keras.Sequential([
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X3_2_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model3_2.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model3_2 = reg_model3_2.fit(
    X3_2_train_scaled, y3_2_train_final,
    validation_data=(X3_2_val_scaled, y3_2_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# model evaluation
print("MODEL EVALUATION: MAX PEAK POSITION, WITH GENRE")

print("\n=== Baseline Model ===")
train_scores3_2 = baseline_model3_2.evaluate(X3_2_train_scaled, y3_2_train_final, verbose=0)
val_scores3_2   = baseline_model3_2.evaluate(X3_2_val_scaled, y3_2_val, verbose=0)
print(f"Train MAE: {train_scores3_2[1]:.4f}, Train MSE: {train_scores3_2[2]:.4f}")
print(f"Val   MAE: {val_scores3_2[1]:.4f}, Val   MSE: {val_scores3_2[2]:.4f}")

print("\n=== BatchNorm Model ===")
train_scores_bn3_2 = bnorm_model3_2.evaluate(X3_2_train_scaled, y3_2_train_final, verbose=0)
val_scores_bn3_2   = bnorm_model3_2.evaluate(X3_2_val_scaled, y3_2_val, verbose=0)
print(f"Train MAE: {train_scores_bn3_2[1]:.4f}, Train MSE: {train_scores_bn3_2[2]:.4f}")
print(f"Val   MAE: {val_scores_bn3_2[1]:.4f}, Val   MSE: {val_scores_bn3_2[2]:.4f}")

print("\n=== Regularized Model (L2 + Dropout) ===")
train_scores_reg3_2 = reg_model3_2.evaluate(X3_2_train_scaled, y3_2_train_final, verbose=0)
val_scores_reg3_2   = reg_model3_2.evaluate(X3_2_val_scaled, y3_2_val, verbose=0)
print(f"Train MAE: {train_scores_reg3_2[1]:.4f}, Train MSE: {train_scores_reg3_2[2]:.4f}")
print(f"Val   MAE: {val_scores_reg3_2[1]:.4f}, Val   MSE: {val_scores_reg3_2[2]:.4f}")


**Deep Learning | Max Rank Change - No Genre**

In [ ]:
# deep learning model, no regularization or dropout

baseline_model4_1 = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X4_1_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Single output for regression
])

baseline_model4_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_baseline4_1 = baseline_model4_1.fit(
    X4_1_train_scaled, y4_1_train_final,
    validation_data=(X4_1_val_scaled, y4_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# deep learning model with batch normalization

bnorm_model4_1 = keras.Sequential([
    layers.Dense(64, activation='linear', input_shape=(X4_1_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),

    layers.Dense(1)  # Single output for regression
])

bnorm_model4_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_bnorm_model4_1 = bnorm_model4_1.fit(
    X4_1_train_scaled, y4_1_train_final,
    validation_data=(X4_1_val_scaled, y4_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# deep learning model with regularization (L2 and dropout)

l2_reg = 1e-4
dropout_rate = 0.4

reg_model4_1 = keras.Sequential([
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X4_1_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model4_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model4_1 = reg_model4_1.fit(
    X4_1_train_scaled, y4_1_train_final,
    validation_data=(X4_1_val_scaled, y4_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# model evaluation
print("MODEL EVALUATION: MAX RANK CHANGE, NO GENRE")

print("\n=== Baseline Model ===")
train_scores4_1 = baseline_model4_1.evaluate(X4_1_train_scaled, y4_1_train_final, verbose=0)
val_scores4_1   = baseline_model4_1.evaluate(X4_1_val_scaled, y4_1_val, verbose=0)
print(f"Train MAE: {train_scores4_1[1]:.4f}, Train MSE: {train_scores4_1[2]:.4f}")
print(f"Val   MAE: {val_scores4_1[1]:.4f}, Val   MSE: {val_scores4_1[2]:.4f}")

print("\n=== BatchNorm Model ===")
train_scores_bn4_1 = bnorm_model4_1.evaluate(X4_1_train_scaled, y4_1_train_final, verbose=0)
val_scores_bn4_1   = bnorm_model4_1.evaluate(X4_1_val_scaled, y4_1_val, verbose=0)
print(f"Train MAE: {train_scores_bn4_1[1]:.4f}, Train MSE: {train_scores_bn4_1[2]:.4f}")
print(f"Val   MAE: {val_scores_bn4_1[1]:.4f}, Val   MSE: {val_scores_bn4_1[2]:.4f}")

print("\n=== Regularized Model (L2 + Dropout) ===")
train_scores_reg4_1 = reg_model4_1.evaluate(X4_1_train_scaled, y4_1_train_final, verbose=0)
val_scores_reg4_1   = reg_model4_1.evaluate(X4_1_val_scaled, y4_1_val, verbose=0)
print(f"Train MAE: {train_scores_reg4_1[1]:.4f}, Train MSE: {train_scores_reg4_1[2]:.4f}")
print(f"Val   MAE: {val_scores_reg4_1[1]:.4f}, Val   MSE: {val_scores_reg4_1[2]:.4f}")


**Deep Learning | Max Rank Change - With Genre**

In [ ]:
# deep learning model, no regularization or dropout

baseline_model4_2 = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X4_2_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Single output for regression
])

baseline_model4_2.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_baseline4_2 = baseline_model4_2.fit(
    X4_2_train_scaled, y4_2_train_final,
    validation_data=(X4_2_val_scaled, y4_2_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# deep learning model with batch normalization

bnorm_model4_2 = keras.Sequential([
    layers.Dense(64, activation='linear', input_shape=(X4_2_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    
    layers.Dense(64, activation='linear'),
    layers.BatchNormalization(),
    layers.Activation('relu'),

    layers.Dense(1)  # Single output for regression
])

bnorm_model4_2.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_bnorm_model4_2 = bnorm_model4_2.fit(
    X4_2_train_scaled, y4_2_train_final,
    validation_data=(X4_2_val_scaled, y4_2_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# deep learning model with regularization (L2 and dropout)

l2_reg = 1e-4
dropout_rate = 0.4

reg_model4_2 = keras.Sequential([
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X4_2_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model4_2.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model4_2 = reg_model4_2.fit(
    X4_2_train_scaled, y4_2_train_final,
    validation_data=(X4_2_val_scaled, y4_2_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# model evaluation
print("MODEL EVALUATION: MAX RANK CHANGE, WITH GENRE")

print("\n=== Baseline Model ===")
train_scores4_2 = baseline_model4_2.evaluate(X4_2_train_scaled, y4_2_train_final, verbose=0)
val_scores4_2   = baseline_model4_2.evaluate(X4_2_val_scaled, y4_2_val, verbose=0)
print(f"Train MAE: {train_scores4_2[1]:.4f}, Train MSE: {train_scores4_2[2]:.4f}")
print(f"Val   MAE: {val_scores4_2[1]:.4f}, Val   MSE: {val_scores4_2[2]:.4f}")

print("\n=== BatchNorm Model ===")
train_scores_bn4_2 = bnorm_model4_2.evaluate(X4_2_train_scaled, y4_2_train_final, verbose=0)
val_scores_bn4_2   = bnorm_model4_2.evaluate(X4_2_val_scaled, y4_2_val, verbose=0)
print(f"Train MAE: {train_scores_bn4_1[1]:.4f}, Train MSE: {train_scores_bn4_1[2]:.4f}")
print(f"Val   MAE: {val_scores_bn4_1[1]:.4f}, Val   MSE: {val_scores_bn4_1[2]:.4f}")

print("\n=== Regularized Model (L2 + Dropout) ===")
train_scores_reg4_2 = reg_model4_2.evaluate(X4_2_train_scaled, y4_2_train_final, verbose=0)
val_scores_reg4_2   = reg_model4_2.evaluate(X4_2_val_scaled, y4_2_val, verbose=0)
print(f"Train MAE: {train_scores_reg4_2[1]:.4f}, Train MSE: {train_scores_reg4_2[2]:.4f}")
print(f"Val   MAE: {val_scores_reg4_2[1]:.4f}, Val   MSE: {val_scores_reg4_2[2]:.4f}")


**Deep Learning | Optimizing Regularized Models**

Max Peak Position

In [ ]:
# adding a layer

l2_reg = 1e-4
dropout_rate = 0.4

reg_model3_1 = keras.Sequential([
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X3_1_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model3_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model3_1 = reg_model3_1.fit(
    X3_1_train_scaled, y3_1_train_final,
    validation_data=(X3_1_val_scaled, y3_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
print("\n=== Regularized Model (L2 + Dropout) ===")
print("    Adding an additional deep layer")
train_scores_reg3_1 = reg_model3_1.evaluate(X3_1_train_scaled, y3_1_train_final, verbose=0)
val_scores_reg3_1   = reg_model3_1.evaluate(X3_1_val_scaled, y3_1_val, verbose=0)
print(f"Train MAE: {train_scores_reg3_1[1]:.4f}, Train MSE: {train_scores_reg3_1[2]:.4f}")
print(f"Val   MAE: {val_scores_reg3_1[1]:.4f}, Val   MSE: {val_scores_reg3_1[2]:.4f}")


Adding a layer resulted in a larger MAE for both training and validation. Removing the additional layer and adding kernels.

In [ ]:
# adding a layer

l2_reg = 1e-4
dropout_rate = 0.4

reg_model3_1 = keras.Sequential([
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X3_1_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model3_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model3_1 = reg_model3_1.fit(
    X3_1_train_scaled, y3_1_train_final,
    validation_data=(X3_1_val_scaled, y3_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
print("\n=== Regularized Model (L2 + Dropout) ===")
print("Reverting to 2 deep layers, increasing to 128 kernels per layer")
train_scores_reg3_1 = reg_model3_1.evaluate(X3_1_train_scaled, y3_1_train_final, verbose=0)
val_scores_reg3_1   = reg_model3_1.evaluate(X3_1_val_scaled, y3_1_val, verbose=0)
print(f"Train MAE: {train_scores_reg3_1[1]:.4f}, Train MSE: {train_scores_reg3_1[2]:.4f}")
print(f"Val   MAE: {val_scores_reg3_1[1]:.4f}, Val   MSE: {val_scores_reg3_1[2]:.4f}")


Increasing kernels per layer gave a result very similar to the original model but increased overfitting a little bit. Leaving 128 kernels per layer and increasing the dropout rate.

In [ ]:
# increasing dropout rate

l2_reg = 1e-4
dropout_rate = 0.6

reg_model3_1 = keras.Sequential([
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X3_1_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model3_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model3_1 = reg_model3_1.fit(
    X3_1_train_scaled, y3_1_train_final,
    validation_data=(X3_1_val_scaled, y3_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
print("\n=== Regularized Model (L2 + Dropout) ===")
print("2 deep layers,  kernels per layer, increased dropout from 0.4 to 0.6")
train_scores_reg3_1 = reg_model3_1.evaluate(X3_1_train_scaled, y3_1_train_final, verbose=0)
val_scores_reg3_1   = reg_model3_1.evaluate(X3_1_val_scaled, y3_1_val, verbose=0)
print(f"Train MAE: {train_scores_reg3_1[1]:.4f}, Train MSE: {train_scores_reg3_1[2]:.4f}")
print(f"Val   MAE: {val_scores_reg3_1[1]:.4f}, Val   MSE: {val_scores_reg3_1[2]:.4f}")


In [ ]:
# adding a layer

l2_reg = 1e-4
dropout_rate = 0.6

reg_model3_1 = keras.Sequential([
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X3_1_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model3_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model3_1 = reg_model3_1.fit(
    X3_1_train_scaled, y3_1_train_final,
    validation_data=(X3_1_val_scaled, y3_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
print("\n=== Regularized Model (L2 + Dropout) ===")
print("3 deep layers, 128 kernels per layer, 0.6 dropout rate")
train_scores_reg3_1 = reg_model3_1.evaluate(X3_1_train_scaled, y3_1_train_final, verbose=0)
val_scores_reg3_1   = reg_model3_1.evaluate(X3_1_val_scaled, y3_1_val, verbose=0)
print(f"Train MAE: {train_scores_reg3_1[1]:.4f}, Train MSE: {train_scores_reg3_1[2]:.4f}")
print(f"Val   MAE: {val_scores_reg3_1[1]:.4f}, Val   MSE: {val_scores_reg3_1[2]:.4f}")


The original regularized model continues to have the best MAE.

Max Rank Change

In [ ]:
# adding a layer 

l2_reg = 1e-4
dropout_rate = 0.4

reg_model4_1 = keras.Sequential([
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X4_1_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(64, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model4_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model4_1 = reg_model4_1.fit(
    X4_1_train_scaled, y4_1_train_final,
    validation_data=(X4_1_val_scaled, y4_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
print("=== Regularized Model (L2 + Dropout) ===")
print("    Adding a layer")
train_scores_reg4_1 = reg_model4_1.evaluate(X4_1_train_scaled, y4_1_train_final, verbose=0)
val_scores_reg4_1   = reg_model4_1.evaluate(X4_1_val_scaled, y4_1_val, verbose=0)
print(f"Train MAE: {train_scores_reg4_1[1]:.4f}, Train MSE: {train_scores_reg4_1[2]:.4f}")
print(f"Val   MAE: {val_scores_reg4_1[1]:.4f}, Val   MSE: {val_scores_reg4_1[2]:.4f}")


These results are very nearly identical to the initial configurations. Doubling the kernels in each layer.

In [ ]:
# adding a layer 

l2_reg = 1e-4
dropout_rate = 0.4

reg_model4_1 = keras.Sequential([
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg),
                 input_shape=(X4_1_train.shape[1],)),
    layers.Dropout(dropout_rate),
        
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),

    layers.Dense(1)  # Single output for regression
])

reg_model4_1.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

history_reg_model4_1 = reg_model4_1.fit(
    X4_1_train_scaled, y4_1_train_final,
    validation_data=(X4_1_val_scaled, y4_1_val),
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
print("=== Regularized Model (L2 + Dropout) ===")
print("    Adding a layer")
train_scores_reg4_1 = reg_model4_1.evaluate(X4_1_train_scaled, y4_1_train_final, verbose=0)
val_scores_reg4_1   = reg_model4_1.evaluate(X4_1_val_scaled, y4_1_val, verbose=0)
print(f"Train MAE: {train_scores_reg4_1[1]:.4f}, Train MSE: {train_scores_reg4_1[2]:.4f}")
print(f"Val   MAE: {val_scores_reg4_1[1]:.4f}, Val   MSE: {val_scores_reg4_1[2]:.4f}")

The original regularized model has the best MAE scores.

## Evaluation

### Summary of Model Performance

**XGBoost**
The XGBoost model did not return meaningful results for this dataset. The best metrics for XGBoost were:

_Maximum Peak Position_

- RMSE: 22.35
- r<sup>2</sup>: 0.138

_Maximum Rank Increase_

- RMSE: 11.73
- - r<sup>2</sup>: 0.002

**k-Nearest Neighbors**
This model also did not perform well on this data set. Its best metrics were:

_Maximum Peak Position_

- Accuracy: 0.052

_Maximum Rank Increase_

- Accuracy: 0.217

**Deep Learning**

The deep learning model returned the most promising results and are described in the next section.

### Final Models

The final models are both based on a deep learning architecture. 

_Maximum Peak Position_

The best model for the Maximum Peak Position is named reg_model3_1. It is a 3-layer, regularized model trained on the song dataset that does not contain genre information.Its final metrics were:

- Training mean absolute error: 16.99
- Validation mean absolute error: 17.67

_Maximum Rank Increase_

The best model for the Maximum Rank Increase is named reg_model4_2. It is also a 3-layer, regularized model trained on the song dataset without genre.
Its final metrics:

- Training mean absolute error: 7.75
- Validation mean absolute error: 8.91

### Business Insight/Recommendation 1

The genre of a song, at least how it is captured on Spotify, has very little influence on its movement on the Billboard Hot 100 list. This gives FutureProduct Advisors consultants and their customers significant leeway when they are selecting music for their social and advertising campaigns.

### Business Insight/Recommendation 2

The characteristics of a given song appear to be the most important factors in its performance on the Hot 100 list. Features such as tempo, danceability, etc. have the most predictive power for the Hot 100 list.

### Business Insight/Recommendation 3

Digital music has a large number of characteristics that can be pulled into machine learning models. The factors that have the largest influence on popularity are often unexpected, and modeling and analysis of songs requires methodical and careful selection and vetting of features.

## Conclusion and Next Steps

reg_model3_1 (predicting highest ranking on the Billboard Hot 100) and reg_model4_2 (predicting largest week over week ranking increase) are ready for initial deployment to FutureProduct Advisors. 

Detailed training will be provided, but at a high level here is how users will engage:

1. Consultants will identify a list of songs they'd like to explore.
    - We recommend using the songs in places 90-100 of the Billboard Hot 100 at a minimum
2. Consultants download those songs' metadata from Spotify as a CSV file (detailed instructions to come)
3. Consultants load the CSV file into these models, and the models will return predicted activity for each song.

There is also an additional opportunity to built on and refine these models by indluding additional data and experimenting with other modeling approaches. If the FutureProduct Advisors consultants have positive feedback on these prototype tools, we will be happy to partner on future enhancements.
